In [5]:
#Based off recsys ising starter code

import matplotlib.pyplot as plt
import pyGMs as gm
import numpy as np
import time
import pickle
import numpy as np
import pyGMs.ising

'''
start_time = time.process_time_ns()
time_elasped = time.process_time_ns() - start_time
print(time_elasped/(10**9))
'''

with open('user_dict.pkl', 'rb') as file:
    loaded_data = pickle.load(file)
with open('train_users.pkl', 'rb') as file:
    loaded_training = pickle.load(file)
with open('test_users.pkl', 'rb') as file:
    loaded_test = pickle.load(file)
with open('movie_name.pkl', 'rb') as file:
    movie_name = pickle.load(file)
    
Xtr = np.asarray([loaded_data[x] for x in loaded_training])
Xte = np.asarray([loaded_data[x] for x in loaded_test])

Xtr_missing = np.where(np.isnan(Xtr), -1, Xtr).astype(int)
Xte_missing = np.where(np.isnan(Xte), -1, Xte).astype(int)
Xtr = (Xtr >= 4).astype(int)   # did each user like the movie?  (binary)
Xte = (Xte >= 4).astype(int)

mUsers, nMovies = Xtr.shape

In [6]:
def impute_missing(model, Xobs):
    m,n = Xobs.shape
    Xhat = np.copy(Xobs);
    for j in range(m):
        x_obs = {i:Xobs[j,i] for i in range(n) if Xobs[j,i] in (0,1)}
        x_unobs = [i for i in range(n) if Xobs[j,i] not in (0,1)]
        if not x_unobs: continue
        cond = gm.GraphModel([f.condition(x_obs) for f in model.factorsWithAny(x_unobs)])
        for x in cond.X:
            if x.states == 0: x.states = 1;  # fix a bug in GraphModel behavior for missing vars...
        jt = gm.wmb.JTree(cond, weights=1e-6) # 0: for maximization
        x_hat = jt.argmax();
        for i in x_unobs: Xhat[j,i] = x_hat[i]
    return Xhat

In [7]:
Xte_missing = Xte[:100, :int(0.7*len(Xte[0]))]
miss_x, miss_y = Xte_missing.shape

In [ ]:
train_len =  [500, 1000, 2500, 5040]
c_vals = [0.0001, 0.001, 0.01, 0.1, 0.5, 1, 5, 10, 50, 100]
for x in train_len:
    for y in c_vals:
        file = open("resultsVersion2.txt", "a")
        file.write(f"Results with training length of {x} and C penalty value of {y}\n")
        start_time = time.process_time_ns()
        ising_l1 = gm.ising.fit_logregL1(Xtr[:x], C=y)
        time_elasped = time.process_time_ns() - start_time
        file.write(f"Time taken to train in seconds: {time_elasped/(10**9)}\n")
        gm.drawMarkovGraph(ising_l1);
        plt.savefig(f"Size{x}C{y}V2_1.png")
        file.write(f"Pseudo-likelihood training set: {ising_l1.pseudolikelihood(Xtr[:x]).mean()}\n")
        file.write(f"Pseudo-likelihood testing set: {ising_l1.pseudolikelihood(Xte).mean()}\n\n")
        #Xte_hat = impute_missing(ising_l1, Xte_missing)
        #file.write(f'Error rate: {np.mean( (Xte_hat!=Xte_missing)[Xte_hat<0])}\n\n')
        file.close()